# Capstone Project 2: Equity Analysis - Model Performance by Student Subgroups

## Objective
This notebook walks you through the UPAD cycle (Understand, Prepare, Analyze, Deploy) to:
- Analyze model fairness across Race/Ethnicity, First Generation Status, and Gender
- Build and compare best-performing models for each demographic subgroup
- Compare performance metrics across groups to identify potential bias
- Discuss fairness implications and develop higher education policy recommendations

# Understand

## The Importance of Equity in Predictive Analytics

As universities increasingly adopt machine learning models for student success initiatives, it is critical to ensure these models do not perpetuate or amplify existing inequities. A model that performs well on average may still produce systematically different outcomes for different student populations.

**Fairness concerns in higher education include:**
- **Predictive parity**: Does the model predict with equal accuracy across groups?
- **Error rate parity**: Are false positive and false negative rates similar across groups?
- **Disparate impact**: Do model predictions lead to different intervention rates by group?

This capstone project examines model performance across three key demographic dimensions:
1. **Race/Ethnicity**: Ensuring the model works equally well for students of all racial/ethnic backgrounds
2. **First Generation Status**: Checking if first-generation students are fairly served
3. **Gender**: Verifying gender equity in model predictions

### Learning Objectives

By the end of this capstone, you will be able to:
1. Evaluate model performance separately for demographic subgroups
2. Identify potential sources of algorithmic bias
3. Apply fairness metrics to machine learning models
4. Develop policy recommendations for equitable model deployment

# Prepare

## Data Wrangling

#### **Step 1: Import Libraries and Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

# Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report,
    average_precision_score
)

# Set random seed
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("All libraries imported successfully!")

In [ ]:
# Load data
data_location = '/content/drive/MyDrive/projects/Applied-Data-Analytics-For-Higher-Education-Course-2/data/'
df = pd.read_csv(f'{data_location}student_academics_data.csv')
print(f"Dataset shape: {df.shape}")
df.head()

#### **Step 2: Data Cleaning and Preparation**

In [ ]:
# Address Rare Classes in RACE_ETHNICITY
df['RACE_ETHNICITY'] = df['RACE_ETHNICITY'].replace(
    ['Unknown', 'Native Hawaiian or Other Pacific Islander', 'American Indian or Alaska Native'], 
    'Other'
)

# Address Rare Classes in GENDER
df = df[df['GENDER'] != 'Nonbinary']
df['GENDER'] = df['GENDER'].str.strip().str.capitalize()

# Drop noninformative features
df.drop(['SEM_1_STATUS', 'SEM_2_STATUS'], axis=1, inplace=True)

# Remove duplicates
df.drop_duplicates(inplace=True)

# Drop columns with >50% missing values
missing_pct = df.isnull().sum() / len(df)
cols_to_drop = missing_pct[missing_pct > 0.5].index.tolist()
df.drop(columns=cols_to_drop, inplace=True)

# Create binary target
df['DEPARTED'] = (df['SEM_3_STATUS'] != 'E').astype(int)

print(f"Cleaned dataset shape: {df.shape}")
print(f"\nDeparture rate: {df['DEPARTED'].mean():.2%}")

#### **Step 3: Examine Demographic Distributions**

In [ ]:
# Display demographic distributions
print("="*60)
print("DEMOGRAPHIC DISTRIBUTIONS")
print("="*60)

print("\nRace/Ethnicity:")
print(df['RACE_ETHNICITY'].value_counts())

print("\nFirst Generation Status:")
print(df['FIRST_GEN_STATUS'].value_counts())

print("\nGender:")
print(df['GENDER'].value_counts())

In [ ]:
# Visualize departure rates by demographic groups
fig = make_subplots(rows=1, cols=3, 
                    subplot_titles=('By Race/Ethnicity', 'By First Gen Status', 'By Gender'))

# Race/Ethnicity
race_rates = df.groupby('RACE_ETHNICITY')['DEPARTED'].agg(['mean', 'count']).reset_index()
race_rates.columns = ['Group', 'Departure Rate', 'N']
race_rates = race_rates.sort_values('Departure Rate', ascending=False)

fig.add_trace(
    go.Bar(x=race_rates['Group'], y=race_rates['Departure Rate'], 
           text=[f"{r:.1%}<br>n={n:,}" for r, n in zip(race_rates['Departure Rate'], race_rates['N'])],
           textposition='outside', marker_color='steelblue'),
    row=1, col=1
)

# First Gen Status
fg_rates = df.groupby('FIRST_GEN_STATUS')['DEPARTED'].agg(['mean', 'count']).reset_index()
fg_rates.columns = ['Group', 'Departure Rate', 'N']

fig.add_trace(
    go.Bar(x=fg_rates['Group'], y=fg_rates['Departure Rate'],
           text=[f"{r:.1%}<br>n={n:,}" for r, n in zip(fg_rates['Departure Rate'], fg_rates['N'])],
           textposition='outside', marker_color='coral'),
    row=1, col=2
)

# Gender
gender_rates = df.groupby('GENDER')['DEPARTED'].agg(['mean', 'count']).reset_index()
gender_rates.columns = ['Group', 'Departure Rate', 'N']

fig.add_trace(
    go.Bar(x=gender_rates['Group'], y=gender_rates['Departure Rate'],
           text=[f"{r:.1%}<br>n={n:,}" for r, n in zip(gender_rates['Departure Rate'], gender_rates['N'])],
           textposition='outside', marker_color='forestgreen'),
    row=1, col=3
)

fig.update_layout(
    title='Departure Rates by Demographic Group',
    height=450,
    showlegend=False
)
fig.update_xaxes(tickangle=-45)
fig.update_yaxes(tickformat='.0%')

fig.show()

#### **Step 4: Prepare Features and Train/Test Split**

In [ ]:
# Feature engineering
def create_features(df):
    df = df.copy()
    df['DFW_RATE_1'] = ((df['UNITS_ATTEMPTED_1'] - df['UNITS_COMPLETED_1']).clip(lower=0) 
                        / df['UNITS_ATTEMPTED_1'].replace(0, 1))
    df['DFW_RATE_2'] = ((df['UNITS_ATTEMPTED_2'] - df['UNITS_COMPLETED_2']).clip(lower=0) 
                        / df['UNITS_ATTEMPTED_2'].replace(0, 1))
    df['GRADE_POINTS_1'] = df['UNITS_ATTEMPTED_1'] * df['GPA_1']
    df['GRADE_POINTS_2'] = df['UNITS_ATTEMPTED_2'] * df['GPA_2']
    return df

df = create_features(df)

In [ ]:
# Define features
numeric_features = [
    'HS_GPA', 'HS_MATH_GPA', 'HS_ENGL_GPA',
    'UNITS_ATTEMPTED_1', 'UNITS_ATTEMPTED_2',
    'UNITS_COMPLETED_1', 'UNITS_COMPLETED_2',
    'DFW_UNITS_1', 'DFW_UNITS_2',
    'GPA_1', 'GPA_2',
    'DFW_RATE_1', 'DFW_RATE_2',
    'GRADE_POINTS_1', 'GRADE_POINTS_2'
]

categorical_features = ['COLLEGE']  # Note: We exclude demographic features from model inputs

# Store demographic columns for later analysis
demographic_cols = ['RACE_ETHNICITY', 'FIRST_GEN_STATUS', 'GENDER']

target = 'DEPARTED'

In [ ]:
# Handle missing values
for col in numeric_features:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].median())

# Train/Test split - maintain demographic information for fairness analysis
train_df, test_df = train_test_split(df, test_size=0.2, random_state=RANDOM_STATE, stratify=df['DEPARTED'])

print(f"Training set: {len(train_df):,} students")
print(f"Testing set: {len(test_df):,} students")

In [ ]:
# Prepare feature matrices
train_encoded = pd.get_dummies(train_df[numeric_features + categorical_features], 
                               columns=categorical_features, drop_first=True)
test_encoded = pd.get_dummies(test_df[numeric_features + categorical_features], 
                              columns=categorical_features, drop_first=True)

# Align columns
train_encoded, test_encoded = train_encoded.align(test_encoded, join='left', axis=1, fill_value=0)

# Fill any remaining NaN
train_encoded = train_encoded.fillna(train_encoded.median())
test_encoded = test_encoded.fillna(test_encoded.median())

X_train = train_encoded
y_train = train_df[target]
X_test = test_encoded
y_test = test_df[target]

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"X_train shape: {X_train.shape}")

# Analyze

## Part 1: Train the Best Model

First, we train a high-performing model on the full dataset, then analyze its performance across subgroups.

#### **Step 5: Train Best-Performing Model**

In [ ]:
# Train a Random Forest (typically performs well and provides feature importance)
print("Training Random Forest model...")

rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=12,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    class_weight='balanced',
    n_jobs=-1,
    random_state=RANDOM_STATE
)
rf_model.fit(X_train, y_train)

# Get predictions
y_pred = rf_model.predict(X_test)
y_prob = rf_model.predict_proba(X_test)[:, 1]

print(f"\nOverall Model Performance:")
print(f"ROC-AUC: {roc_auc_score(y_test, y_prob):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

## Part 2: Fairness Analysis by Subgroup

#### **Step 6: Define Fairness Metrics Function**

In [ ]:
def calculate_subgroup_metrics(y_true, y_pred, y_prob, group_labels, group_name):
    """
    Calculate performance metrics for each subgroup.
    """
    results = []
    
    for group in group_labels.unique():
        mask = group_labels == group
        n = mask.sum()
        
        if n < 10:  # Skip groups with too few samples
            continue
            
        y_true_group = y_true[mask]
        y_pred_group = y_pred[mask]
        y_prob_group = y_prob[mask]
        
        # Calculate confusion matrix components
        tn, fp, fn, tp = confusion_matrix(y_true_group, y_pred_group, labels=[0, 1]).ravel()
        
        metrics = {
            'Group': group,
            'N': n,
            'Base Rate': y_true_group.mean(),  # Actual departure rate
            'Positive Rate': y_pred_group.mean(),  # Predicted departure rate
            'Accuracy': accuracy_score(y_true_group, y_pred_group),
            'Precision': precision_score(y_true_group, y_pred_group, zero_division=0),
            'Recall': recall_score(y_true_group, y_pred_group, zero_division=0),
            'F1 Score': f1_score(y_true_group, y_pred_group, zero_division=0),
            'ROC-AUC': roc_auc_score(y_true_group, y_prob_group) if len(np.unique(y_true_group)) > 1 else np.nan,
            'FPR': fp / (fp + tn) if (fp + tn) > 0 else 0,  # False Positive Rate
            'FNR': fn / (fn + tp) if (fn + tp) > 0 else 0,  # False Negative Rate
        }
        results.append(metrics)
    
    return pd.DataFrame(results)

#### **Step 7: Analyze Performance by Race/Ethnicity**

In [ ]:
# Get test set demographic labels
test_race = test_df['RACE_ETHNICITY'].values

# Calculate metrics by race/ethnicity
race_metrics = calculate_subgroup_metrics(
    y_test.values, y_pred, y_prob, 
    pd.Series(test_race), 'Race/Ethnicity'
)

print("="*100)
print("MODEL PERFORMANCE BY RACE/ETHNICITY")
print("="*100)
display_cols = ['Group', 'N', 'Base Rate', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC-AUC', 'FPR', 'FNR']
print(race_metrics[display_cols].round(4).to_string(index=False))
print("="*100)

In [ ]:
# Visualize racial equity metrics
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=('ROC-AUC by Race/Ethnicity', 'F1 Score by Race/Ethnicity',
                                   'False Positive Rate by Race/Ethnicity', 'False Negative Rate by Race/Ethnicity'))

race_metrics_sorted = race_metrics.sort_values('N', ascending=False)

# ROC-AUC
fig.add_trace(
    go.Bar(x=race_metrics_sorted['Group'], y=race_metrics_sorted['ROC-AUC'], marker_color='steelblue'),
    row=1, col=1
)

# F1 Score
fig.add_trace(
    go.Bar(x=race_metrics_sorted['Group'], y=race_metrics_sorted['F1 Score'], marker_color='forestgreen'),
    row=1, col=2
)

# False Positive Rate
fig.add_trace(
    go.Bar(x=race_metrics_sorted['Group'], y=race_metrics_sorted['FPR'], marker_color='coral'),
    row=2, col=1
)

# False Negative Rate
fig.add_trace(
    go.Bar(x=race_metrics_sorted['Group'], y=race_metrics_sorted['FNR'], marker_color='purple'),
    row=2, col=2
)

fig.update_layout(height=700, title_text="Equity Analysis: Model Performance by Race/Ethnicity", showlegend=False)
fig.update_xaxes(tickangle=-45)
fig.show()

#### **Step 8: Analyze Performance by First Generation Status**

In [ ]:
# Get test set first gen labels
test_firstgen = test_df['FIRST_GEN_STATUS'].values

# Calculate metrics by first gen status
fg_metrics = calculate_subgroup_metrics(
    y_test.values, y_pred, y_prob, 
    pd.Series(test_firstgen), 'First Gen Status'
)

print("="*100)
print("MODEL PERFORMANCE BY FIRST GENERATION STATUS")
print("="*100)
print(fg_metrics[display_cols].round(4).to_string(index=False))
print("="*100)

In [ ]:
# Visualize first gen equity metrics
fig = go.Figure()

metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC-AUC']
colors = ['steelblue', 'coral', 'forestgreen', 'purple', 'orange']

for i, metric in enumerate(metrics_to_plot):
    fig.add_trace(go.Bar(
        name=metric,
        x=fg_metrics['Group'],
        y=fg_metrics[metric],
        marker_color=colors[i]
    ))

fig.update_layout(
    title='Equity Analysis: Model Performance by First Generation Status',
    barmode='group',
    height=450,
    legend=dict(orientation='h', y=1.1)
)
fig.show()

#### **Step 9: Analyze Performance by Gender**

In [ ]:
# Get test set gender labels
test_gender = test_df['GENDER'].values

# Calculate metrics by gender
gender_metrics = calculate_subgroup_metrics(
    y_test.values, y_pred, y_prob, 
    pd.Series(test_gender), 'Gender'
)

print("="*100)
print("MODEL PERFORMANCE BY GENDER")
print("="*100)
print(gender_metrics[display_cols].round(4).to_string(index=False))
print("="*100)

In [ ]:
# Visualize gender equity metrics
fig = go.Figure()

for i, metric in enumerate(metrics_to_plot):
    fig.add_trace(go.Bar(
        name=metric,
        x=gender_metrics['Group'],
        y=gender_metrics[metric],
        marker_color=colors[i]
    ))

fig.update_layout(
    title='Equity Analysis: Model Performance by Gender',
    barmode='group',
    height=400,
    legend=dict(orientation='h', y=1.1)
)
fig.show()

## Part 3: Fairness Disparity Analysis

#### **Step 10: Calculate Disparity Ratios**

In [ ]:
def calculate_disparity_ratios(metrics_df, metric_name, reference_group=None):
    """
    Calculate disparity ratios relative to a reference group.
    A ratio of 1.0 indicates parity.
    """
    if reference_group is None:
        # Use group with largest N as reference
        reference_group = metrics_df.loc[metrics_df['N'].idxmax(), 'Group']
    
    reference_value = metrics_df.loc[metrics_df['Group'] == reference_group, metric_name].values[0]
    
    disparity = metrics_df.copy()
    disparity[f'{metric_name} Ratio'] = disparity[metric_name] / reference_value
    
    return disparity[['Group', 'N', metric_name, f'{metric_name} Ratio']], reference_group

In [ ]:
# Calculate disparity ratios for key metrics
print("="*80)
print("FAIRNESS DISPARITY ANALYSIS")
print("="*80)

# ROC-AUC disparity by race
auc_disparity, ref = calculate_disparity_ratios(race_metrics, 'ROC-AUC')
print(f"\nROC-AUC Disparity by Race/Ethnicity (Reference: {ref})")
print(auc_disparity.round(4).to_string(index=False))

# FPR disparity by race (lower is better, so we want ratios close to 1)
fpr_disparity, ref = calculate_disparity_ratios(race_metrics, 'FPR')
print(f"\nFalse Positive Rate Disparity by Race/Ethnicity (Reference: {ref})")
print(fpr_disparity.round(4).to_string(index=False))

# FNR disparity by race
fnr_disparity, ref = calculate_disparity_ratios(race_metrics, 'FNR')
print(f"\nFalse Negative Rate Disparity by Race/Ethnicity (Reference: {ref})")
print(fnr_disparity.round(4).to_string(index=False))

#### **Step 11: Visualize Comprehensive Fairness Dashboard**

In [ ]:
# Create comprehensive fairness dashboard
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'ROC-AUC by Race/Ethnicity', 'Error Rates by Race/Ethnicity',
        'ROC-AUC by First Gen Status', 'Error Rates by First Gen Status',
        'ROC-AUC by Gender', 'Error Rates by Gender'
    )
)

# Race/Ethnicity ROC-AUC
fig.add_trace(
    go.Bar(x=race_metrics_sorted['Group'], y=race_metrics_sorted['ROC-AUC'], 
           marker_color='steelblue', name='ROC-AUC'),
    row=1, col=1
)

# Race/Ethnicity Error Rates
fig.add_trace(
    go.Bar(x=race_metrics_sorted['Group'], y=race_metrics_sorted['FPR'], 
           name='FPR', marker_color='coral'),
    row=1, col=2
)
fig.add_trace(
    go.Bar(x=race_metrics_sorted['Group'], y=race_metrics_sorted['FNR'], 
           name='FNR', marker_color='purple'),
    row=1, col=2
)

# First Gen ROC-AUC
fig.add_trace(
    go.Bar(x=fg_metrics['Group'], y=fg_metrics['ROC-AUC'], 
           marker_color='steelblue', showlegend=False),
    row=2, col=1
)

# First Gen Error Rates
fig.add_trace(
    go.Bar(x=fg_metrics['Group'], y=fg_metrics['FPR'], 
           marker_color='coral', showlegend=False),
    row=2, col=2
)
fig.add_trace(
    go.Bar(x=fg_metrics['Group'], y=fg_metrics['FNR'], 
           marker_color='purple', showlegend=False),
    row=2, col=2
)

# Gender ROC-AUC
fig.add_trace(
    go.Bar(x=gender_metrics['Group'], y=gender_metrics['ROC-AUC'], 
           marker_color='steelblue', showlegend=False),
    row=3, col=1
)

# Gender Error Rates
fig.add_trace(
    go.Bar(x=gender_metrics['Group'], y=gender_metrics['FPR'], 
           marker_color='coral', showlegend=False),
    row=3, col=2
)
fig.add_trace(
    go.Bar(x=gender_metrics['Group'], y=gender_metrics['FNR'], 
           marker_color='purple', showlegend=False),
    row=3, col=2
)

fig.update_layout(
    height=900,
    title_text="Comprehensive Fairness Dashboard: Model Performance Across Demographics",
    barmode='group'
)
fig.update_xaxes(tickangle=-45)

fig.show()

## Part 4: Subgroup-Specific Models

#### **Step 12: Train Separate Models for Key Subgroups**

In [ ]:
def train_subgroup_model(train_df, test_df, subgroup_col, subgroup_val, X_cols, target):
    """
    Train a model specifically for a demographic subgroup.
    """
    # Filter to subgroup
    train_sub = train_df[train_df[subgroup_col] == subgroup_val].copy()
    test_sub = test_df[test_df[subgroup_col] == subgroup_val].copy()
    
    if len(train_sub) < 100 or len(test_sub) < 20:
        return None, None, None
    
    # Prepare features
    X_train_sub = train_sub[X_cols].fillna(train_sub[X_cols].median())
    y_train_sub = train_sub[target]
    X_test_sub = test_sub[X_cols].fillna(train_sub[X_cols].median())
    y_test_sub = test_sub[target]
    
    # Train model
    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        class_weight='balanced',
        random_state=RANDOM_STATE,
        n_jobs=-1
    )
    model.fit(X_train_sub, y_train_sub)
    
    # Evaluate
    y_pred_sub = model.predict(X_test_sub)
    y_prob_sub = model.predict_proba(X_test_sub)[:, 1]
    
    metrics = {
        'Subgroup': subgroup_val,
        'N_train': len(train_sub),
        'N_test': len(test_sub),
        'Accuracy': accuracy_score(y_test_sub, y_pred_sub),
        'F1 Score': f1_score(y_test_sub, y_pred_sub),
        'ROC-AUC': roc_auc_score(y_test_sub, y_prob_sub) if len(np.unique(y_test_sub)) > 1 else np.nan
    }
    
    return model, metrics, (y_test_sub, y_pred_sub, y_prob_sub)

In [ ]:
# Train models for first gen vs continuing gen
print("Training subgroup-specific models...")
print("="*80)

X_cols = numeric_features
subgroup_results = []

for fg_status in ['First Generation', 'Continuing Generation']:
    model, metrics, _ = train_subgroup_model(
        train_df, test_df, 'FIRST_GEN_STATUS', fg_status, X_cols, target
    )
    if metrics:
        subgroup_results.append(metrics)
        print(f"\n{fg_status}:")
        print(f"  Training samples: {metrics['N_train']:,}")
        print(f"  Test samples: {metrics['N_test']:,}")
        print(f"  ROC-AUC: {metrics['ROC-AUC']:.4f}")
        print(f"  F1 Score: {metrics['F1 Score']:.4f}")

In [ ]:
# Compare subgroup-specific models vs global model
print("\n" + "="*80)
print("COMPARISON: SUBGROUP MODELS vs GLOBAL MODEL")
print("="*80)

comparison_data = []

for fg_status in ['First Generation', 'Continuing Generation']:
    # Get subgroup-specific model performance
    subgroup_metrics = next((m for m in subgroup_results if m['Subgroup'] == fg_status), None)
    
    # Get global model performance on this subgroup
    global_metrics = fg_metrics[fg_metrics['Group'] == fg_status].iloc[0] if len(fg_metrics[fg_metrics['Group'] == fg_status]) > 0 else None
    
    if subgroup_metrics and global_metrics is not None:
        comparison_data.append({
            'Subgroup': fg_status,
            'Global Model ROC-AUC': global_metrics['ROC-AUC'],
            'Subgroup Model ROC-AUC': subgroup_metrics['ROC-AUC'],
            'Improvement': subgroup_metrics['ROC-AUC'] - global_metrics['ROC-AUC']
        })

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.round(4).to_string(index=False))

# Deploy

#### **Step 13: Generate Equity Report Summary**

In [ ]:
# Calculate summary statistics
race_auc_range = race_metrics['ROC-AUC'].max() - race_metrics['ROC-AUC'].min()
race_fpr_range = race_metrics['FPR'].max() - race_metrics['FPR'].min()
race_fnr_range = race_metrics['FNR'].max() - race_metrics['FNR'].min()

print("="*80)
print("EQUITY ANALYSIS EXECUTIVE SUMMARY")
print("="*80)
print(f"""
OVERVIEW:
This analysis examined model fairness across three demographic dimensions:
- Race/Ethnicity ({len(race_metrics)} groups)
- First Generation Status ({len(fg_metrics)} groups)
- Gender ({len(gender_metrics)} groups)

KEY FINDINGS:

1. RACE/ETHNICITY DISPARITIES:
   - ROC-AUC range: {race_auc_range:.4f} (Max: {race_metrics['ROC-AUC'].max():.4f}, Min: {race_metrics['ROC-AUC'].min():.4f})
   - False Positive Rate range: {race_fpr_range:.4f}
   - False Negative Rate range: {race_fnr_range:.4f}
   
2. FIRST GENERATION STATUS:
   - First Gen ROC-AUC: {fg_metrics[fg_metrics['Group']=='First Generation']['ROC-AUC'].values[0]:.4f if len(fg_metrics[fg_metrics['Group']=='First Generation']) > 0 else 'N/A'}
   - Continuing Gen ROC-AUC: {fg_metrics[fg_metrics['Group']=='Continuing Generation']['ROC-AUC'].values[0]:.4f if len(fg_metrics[fg_metrics['Group']=='Continuing Generation']) > 0 else 'N/A'}

3. GENDER:
   - Female ROC-AUC: {gender_metrics[gender_metrics['Group']=='Female']['ROC-AUC'].values[0]:.4f if len(gender_metrics[gender_metrics['Group']=='Female']) > 0 else 'N/A'}
   - Male ROC-AUC: {gender_metrics[gender_metrics['Group']=='Male']['ROC-AUC'].values[0]:.4f if len(gender_metrics[gender_metrics['Group']=='Male']) > 0 else 'N/A'}
""")
print("="*80)

#### **Step 14: Produce Comprehensive Policy Report**

### Deliverable: Equity Analysis and Policy Recommendations Report

Using the analyses above, write a comprehensive report that addresses the following:

1. **Fairness Metrics Summary**: Create tables showing model performance metrics (Accuracy, Precision, Recall, F1, ROC-AUC, FPR, FNR) for each demographic subgroup. Identify which groups experience the best and worst model performance.

2. **Disparity Analysis**: Calculate and interpret disparity ratios for key metrics. Are there statistically significant differences in model performance across groups? What are the practical implications?

3. **Bias Identification**: Based on the error rate analysis (FPR and FNR), identify potential sources of algorithmic bias:
   - Which groups are more likely to be incorrectly flagged as at-risk (false positives)?
   - Which groups are more likely to be missed by the model (false negatives)?

4. **Root Cause Analysis**: Discuss potential reasons for performance disparities:
   - Training data representation
   - Feature availability and quality
   - Historical patterns in the data

5. **Policy Recommendations**: Provide specific recommendations for:
   - Model deployment considerations (should different thresholds be used for different groups?)
   - Intervention design (how should resources be allocated across groups?)
   - Monitoring and accountability (how should fairness be tracked over time?)
   - Data collection improvements

6. **Ethical Considerations**: Discuss the ethical implications of using predictive models for student success, including:
   - Privacy concerns
   - Potential for stigmatization
   - Transparency and explainability
   - Student agency and autonomy

> **Rubric**: Your report should be 3-4 pages and include:
> - Summary tables of performance metrics by demographic group
> - At least 3 visualizations from your fairness analysis
> - Specific, actionable policy recommendations
> - Discussion of ethical considerations and limitations

---

## Your Report (Write Below)

*[Write your comprehensive equity analysis and policy recommendations report here]*

---